# Preliminaries
First, need to register an account with [ECMWF](https://www.ecmwf.int/) and then with [Copernicus Climate Data Store](https://cds.climate.copernicus.eu/). From there, run the following in the terminal:

`pip install cdsapi`

Next, need to add your key information(found on the Copernicus Climate Data Store profile) to `~/.cdsapirc`. To do that, I did

`nano ./cdsapirc`

With the following text:
url: https://cds.climate.copernicus.eu/api
key: <your-api-key>

Also make sure it is readable only by you for security reasons by doing
`chmod 600 ~/.cdsapirc`

With that done, time to check the setup.


In [1]:
import cdsapi

client = cdsapi.Client()
print("Authentication successful!")
  
dataset = "reanalysis-era5-pressure-levels"
request = {
    "product_type": ["reanalysis"],
    "variable": ["geopotential"],
    "year": ["2024"],
    "month": ["03"],
    "day": ["01"],
    "time": ["13:00"],
    "pressure_level": ["1000"],
    "data_format": "netcdf",
    "download_format": "unarchived"
}
client.retrieve(dataset, request).download()


2024-11-19 11:55:07,073 WARNING [2024-11-19T00:00:00] System is in degraded status due to issues affecting the underlaying infrastructure. Please check updated status [here](https://status.ecmwf.int/)
2024-11-19 11:55:07,074 INFO [2024-09-28T00:00:00] **Welcome to the New Climate Data Store (CDS)!** This new system is in its early days of full operations and still undergoing enhancements and fine tuning. Some disruptions are to be expected. Your 
[feedback](https://jira.ecmwf.int/plugins/servlet/desk/portal/1/create/202) is key to improve the user experience on the new CDS for the benefit of everyone. Thank you.
2024-11-19 11:55:07,074 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2024-11-19 11:55:07,075 INFO [2024-09-16T00:00:00] Remember that you need to have an ECMWF account to use the new CDS. **Your old CDS credentials will not work in new CDS!**
2024-11-19 11:55:07,076 WARNING [2024-06-16T00:00:00] CDS A

Authentication successful!


2024-11-19 11:55:07,389 WARNING [2024-10-10T00:00:00] The final validated ERA5 differs from ERA5T from July 2024 until further notice - please refer to our
[Forum announcement](https://forum.ecmwf.int/t/final-validated-era5-product-to-differ-from-era5t-in-july-2024/6685)
for details and watch it for further updates on this.
2024-11-19 11:55:07,391 INFO Request ID is 628a8a7e-fc75-4a98-9f44-4afe69948b29
2024-11-19 11:55:07,538 INFO status has been updated to accepted
2024-11-19 12:01:27,920 INFO status has been updated to successful


'fe2acbf755a9c1c5ad065d4875249c64.nc'

# Creating the batch list of files to download
This was a nightmare and I have tried a bunch of different ways of doing it. 

In [ ]:
# Load metadata from the JSON file
import json

# Path to your JSON file
file_path = "/Users/aallyn/GitHub/lobSDM/Code/cmip6_input_copernicus.json"

# Open and load the JSON file
with open(file_path, "r") as file:
    metadata = json.load(file)
    
# Iterate over requests and download data
for request in metadata["requests"]:
    model = request["model"]
    experiment = request["experiment"]
    variable = request["variable"]
    member_id = request["member_id"]
    area = request["area"]
    years = request["years"]
    
    # Define output filename
    output_file = f"/Users/aallyn/Library/CloudStorage/Box-Box/RES_Data/CMIP6/SSP5_85/RawTmpFiles/{model}_{experiment}_{member_id}_{variable}.nc"
    
    print(f"Requesting {output_file}...")
    try:
        # Submit the request to CDS
        client.retrieve(
            "projections-cmip6",
            {
                "format": "netcdf",
                "temporal_resolution": "monthly",
                "experiment": experiment,
                "model": model,
                "ensemble_member": member_id,
                "variable": variable,
                "area": area,
                "years": years,
                "data_format": "netcdf",
                "download_format": "unarchived"
            },
            output_file
        )
        print(f"Downloaded {output_file}")
    except Exception as e:
        print(f"Failed to download {output_file}: {e}")